# Notebook for scraping missing country data

This notebook was used to do the following:
- scrape the country for institutions from the web using Wikipedia and Google
- save a file of CS institutions that could not be scraped with their information for use in Mturk tasks
- merge the results of the Mturk task and scraping task to create a new version of inst_full_clean.csv

In [20]:
import os, time, requests, re, json, random, csv
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.exc import GeopyError

In [127]:
# get coordinates from wikipedia
def wikipedia_location(row):
    '''
    row: a row from institution_info.data
    return: (location, method)
    '''
    
    if row[institution_info.headers.index("WikiUrl")] == "":
        return "NO_LOCATION_FOUND", "None"
    
    url = row[institution_info.headers.index("WikiUrl")]
    
    attempts = 5
    successful = False
    
    while attempts > 0:
        try:
            page = requests.get(url)
            attempts = 0
            successful = True
        except (requests.exceptions.ConnectionError):
            time.sleep(0.5)
            attempts -= 1
            
    if not successful:
        print("WARNING: Experience ConnectionError", url)
        return "NO_LOCATION_FOUND", "None"
    
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # check if coordinates are given
    lat_element = soup.find("span", attrs={"class": "latitude"})
    lon_element = soup.find("span", attrs={"class": "longitude"})
    coordinates_present = lat_element != None and lon_element != None
    
    # return if coordinates are given
    if coordinates_present:
        location = lat_element.text, lon_element.text
        method = "Wiki-Coordinates"
        return location, method
        
    # check if country is present
    country_element = soup.find("div", attrs={"class": "country-name"})
    country_present = country_element != None
    
    # return if country is present
    if country_present:
        location = country_element.text
        method = "Wiki-Country"
        return location, method
    
    # check if headquarters location is present
    headquarters_element = soup.find("th",string="Headquarters")
    headquarters_present = headquarters_element != None
    
    # return if HQ is present
    if headquarters_present:
        location = headquarters_element.parent.text.replace("Headquarters","")
        method = "Wiki-Headquarters"
        return location, method
    
    # check if location is present
    location_element = soup.find("th",string="Location")
    location_present = location_element != None
    
    # return if HQ is present
    if location_present:
        location = location_element.parent.text.replace("Location","")
        method = "Wiki-Location"
        return location, method
    
    
    location = "NO_LOCATION_FOUND"
    method = "None"
        
    return location, method


# get location from google location
def google_location(driver, row):
    '''
    driver: a selenium driver handle
    row: a row from institution_info.data
    return: (location, method)
    '''
    
    name = row[institution_info.headers.index("DisplayName")]
    url_google = googlify_name(name)
    driver.get(url_google)
    
    try: 
        element = driver.find_element_by_xpath("//*/div[@class='Z0LcW']")
        location = element.text
        method = "Google-Headquarters"
        return location, method
    except:
        pass

    try: 
        element = element = driver.find_element_by_xpath("//*[@data-dtype='d3ifr']/span[@class='LrzXr']")
        location = element.text
        method = "Google-Sidebar"
        return location, method
    except:
        location = "NO_LOCATION_FOUND"
        method = "None"
        return location, method
    
    
# handle needs for different methods to get location
def get_location(driver, row):
    name = row[institution_info.headers.index("DisplayName")]
    if row[institution_info.headers.index("Country")] != "":
        location = row[institution_info.headers.index("Country")]
        method = "Existing"
        country = location
        return (name,location,method,country)

    location, method = wikipedia_location(row)

    if method != "None":
        country = location_to_country(location,method)
        return (name,location,method,country)
    
    location, method = google_location(driver,row)
    
    country = location_to_country(location,method)
    
    return (name,location,method,country)


def googlify_name(name, appendage="headquarters"):
    # clean name for use in query by url
    name = name.replace(' ', '+'); name = name.replace(',', '')
    name = name.replace("'", ''); name = name.replace('.', ''); name = name.replace(':', '')
    name = name.replace(';', '');
    if appendage is not None: name += "+{}".format(appendage)
    return "https://www.google.com/search?q=" + name    

In [128]:
def location_to_country_coordinates(location):
    geolocator = Nominatim(user_agent="my-application")
    try:
        address = geolocator.reverse(", ".join(location), language="en", addressdetails=True)
        country = address.raw["address"]["country"]
    except KeyError:
        print("ERROR_PROCESSING_LOCATION", location)
        country = "ERROR_PROCESSING_LOCATION"
    return country

def location_to_country_address(location):
    location = split_by_case(location)
    geolocator = Nominatim(user_agent="my-application")
    try:
        address = geolocator.geocode(location, language="en", addressdetails=True)
        country = address.raw["address"]["country"]
    except (AttributeError):
        print("ERROR_PROCESSING_LOCATION", location)
        country = "ERROR_PROCESSING_LOCATION"
    return country

def location_to_country(location, method):
    
    if method == "None":
        country = "NO_COUNTRY_FOUND"
        return country
    
    attempts = 10
    
    while attempts > 0:
        try:
            if method == "Wiki-Coordinates":
                country = location_to_country_coordinates(location)
            else:
                country = location_to_country_address(location)
            return country
        except (GeopyError):
            time.sleep(0.1)
            attempts -= 1

    country = "NO_COUNTRY_FOUND"
    return country

def split_by_case(string):
    
    tokens = list()
    next_token_start = 0
    prev_lower = False
    
    for i, char in enumerate(string):
        current_upper = char.isupper()
        if prev_lower and current_upper:
            tokens.append(string[next_token_start:i])
            next_token_start = i
            
        prev_lower = char.islower() or char.isnumeric()
        next
    
    tokens.append(string[next_token_start:])
    output = ", ".join(tokens)
    return output

In [23]:
class InstitutionInfo:
    def __init__(self):
        self.input_filename = "inst_full_clean.csv"
        self.output_filename = "inst_full_clean_countries.csv"
        self.unsaved_rows = 0
        self.load_data()

    def load_data(self):
        self.output_exists = os.path.exists(self.output_filename)
        if self.output_exists:
            with open(self.output_filename, "r") as fh:
                reader = csv.reader(fh, delimiter=",")
                self.headers = next(reader)
                self.data = [line for line in reader]
        else:
            with open(self.input_filename, "r") as fh:
                reader = csv.reader(fh, delimiter=",")
                self.headers = ["Key","DisplayName","Type","Region","Country","Url"]
                self.data = [line for line in reader] 
            self.add_wikipedia_links()
            self.add_column(["" for row in self.data], header = "Location")
            self.add_column(["" for row in self.data], header = "ScrapedCountry")
            self.add_column(["" for row in self.data], header = "Method")
            self.save()        
        
    def add_wikipedia_links(self):
        assert len(self.headers) == 6
        self.headers.append("WikiUrl")
        with open("inst_fullname.csv", "r") as fh:
            reader = csv.reader(fh, delimiter=",")
            wiki_urls_dict = {row[0]: None if len(row) != 5 else row[4] for row in reader}
        for row in self.data:
            assert len(row) == 6
            row.append(wiki_urls_dict[row[self.headers.index("Key")]])
        self.save()
        
    def add_column(self, column_data, header):
        assert len(column_data) == len(self.data)
        new_data = [row + [column_data[i]] for i, row in enumerate(self.data)]
        self.data = new_data
        self.headers.append(header)
        
    def get_column(self, column_name):
        idx = self.headers.index(column_name)
        return [row[idx] for row in self.data]
    
    def head(self):
        print("\t".join(self.headers))
        for row in self.data[:10]:
            print("\t".join(row))
            
    def save(self, output_filename = None):
        if output_filename == None:
            output_filename = self.output_filename
        with open(output_filename,"w") as fh:
            writer = csv.writer(fh,delimiter=",")
            writer.writerow(self.headers)
            writer.writerows(self.data)
        print("finished saving {} rows".format(self.unsaved_rows))
        self.unsaved_rows = 0

    def get_country_info(self):
        start = datetime.now()
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        
        for row in self.data:
            if row[self.headers.index("ScrapedCountry")] != "":
                continue
                
            if self.unsaved_rows >= 50:
                self.save()
                print("Took",round((datetime.now()-start).total_seconds(),2), "seconds")
                start = datetime.now()
                
            name, location, method, country = get_location(driver,row)
            row[self.headers.index("Location")] = location
            row[self.headers.index("ScrapedCountry")] = country
            row[self.headers.index("Method")] = method
            self.unsaved_rows += 1
            
        driver.close()
        self.save()
        print("Took",round((datetime.now()-start).total_seconds(),2), "seconds")
        print("No more rows to fetch")
        
    def reset(self):
        os.remove(self.output_filename)
        self.load_data()
            

institution_info = InstitutionInfo()

In [129]:
# institution_info.get_country_info()

In [146]:
print("Missing location", 
      len([1 for row in institution_info.get_column("Region") if row == "Other"]))
print("Missing type", 
      len([1 for row in institution_info.get_column("Type") if row == "Other"]))
print("Missing location or type", 
      len([1 for row in institution_info.data if 
            row[institution_info.headers.index("Region")] == "Other" or
            row[institution_info.headers.index("Type")] == "Other"]))

Missing location 6803
Missing type 7769
Missing location or type 7772


In [147]:
# prevent north american region being interpreted as missing
na_vals = [
    '-1.#IND', '1.#QNAN', '1.#IND',
    '-1.#QNAN', '#N/A','N/A',# 'NA',
    '#NA', 'NULL', 'NaN', '-NaN', 
    'nan', '-nan']

full_data = pd.read_csv("inst_full_clean_countries.csv", keep_default_na=False, na_values = na_vals)

In [148]:
MAG_countries = sorted(set(full_data["Country"][np.vectorize(type)(full_data["Country"]) == str]))
scraped_counries = sorted(set(full_data["ScrapedCountry"][full_data["Method"] != "existing"]))

print("COUNTRIES NOT ALREADY IN MAG LIST:\n")
for country in scraped_counries:
    if country not in MAG_countries:
        print(country)

COUNTRIES NOT ALREADY IN MAG LIST:

Angola
Anguilla
Barbados
Czech Republic
Côte d'Ivoire
D.R.
DR Congo
ERROR_PROCESSING_LOCATION
Honduras
Marshall Islands
NO_COUNTRY_FOUND
PRC
Palestinian Territory
RSA
Rwanda
TW
The Netherlands
USA
United States of America
United States of America (Dry Tortugas territorial waters)
Vanuatu


In [149]:
# convert scraped countries to MAG equivalents
country_aliases = {
    "Czech Republic": "Czechia",
    "Côte d'Ivoire": "Ivory Coast",
    "D.R.": "Dominican Republic",
    "DR Congo": "Democratic Republic of the Congo",
    "PRC": "China",
    "Palestinian Territory": "State of Palestine",
    "RSA": "South Africa",
    "TW": "Taiwan",
    "The Netherlands": "Netherlands",
    "USA": "United States",
    "United States of America": "United States",
    "United States of America (Dry Tortugas territorial waters)": "United States",
    "ERROR_PROCESSING_LOCATION": "No data",
    "NO_COUNTRY_FOUND": "No data"
}

country_func = np.vectorize(
    lambda x: country_aliases[x] if x in country_aliases else x)
full_data["FinalCountry"] = country_func(full_data["ScrapedCountry"])

In [150]:
# Get instititions that have scores in CS venues used in CS metrics

# dirname = "../../data/scores/"
# institutions = set()

# for file in os.listdir(dirname):
#     if not file.endswith(".json"):
#         continue
    
#     with open(os.path.join(dirname,file), "r") as fh:
#         institutions.update(json.load(fh).keys())

# institutions = list(institutions)
# with open("institutions_with_scores.json", "w") as fh:
#     json.dump(institutions, fh)

with open("institutions_with_scores.json", "r") as fh:
    institutions = set(json.load(fh))
    
    
has_score_f = lambda x: x in institutions
has_score_f_vec = np.vectorize(has_score_f)
data = full_data[has_score_f_vec(full_data["Key"])]

In [151]:
def print_scraping_summary(data):
    existing = sum(data["Method"] == "Existing")
    not_scraped = sum(data["Method"] == "None")
    condition = np.vectorize(lambda x,y: (x not in ["None","Existing"]) and (y in ["ERROR_PROCESSING_LOCATION","NO_COUNTRY_FOUND"]))
    scraped_not_parsed = sum(condition(data["Method"],data["ScrapedCountry"]))
    condition = np.vectorize(lambda x,y: (x not in ["None","Existing"]) and (y not in ["ERROR_PROCESSING_LOCATION","NO_COUNTRY_FOUND"]))
    scraped = sum(condition(data["Method"],data["ScrapedCountry"]))
    total = len(data)
    assert total == existing + not_scraped + scraped_not_parsed + scraped
    
    print("total:",total)
    print("existing:",existing, " {}%".format(round(100*existing/total,2)))
    print("not scraped:",not_scraped, " {}%".format(round(100*not_scraped/total,2)))
    print("scraped not parsed:",scraped_not_parsed, " {}%".format(round(100*scraped_not_parsed/total,2)))
    print("scraped:",scraped, " {}%".format(round(100*scraped/total,2)))
    print()

print("full data:")
print_scraping_summary(full_data)
print("relevant data:")
print_scraping_summary(data)

full data:
total: 25523
existing: 18720  73.35%
not scraped: 901  3.53%
scraped not parsed: 686  2.69%
scraped: 5216  20.44%

relevant data:
total: 6236
existing: 5094  81.69%
not scraped: 112  1.8%
scraped not parsed: 98  1.57%
scraped: 932  14.95%



In [172]:
# # link countries to regions
with open("country_continent.csv", "r") as fh:
    reader = csv.reader(fh)
    next(reader)
    country_to_region = {country: region for country, region in reader}

In [173]:
country_to_region.keys()

dict_keys(['Andorra', 'United Arab Emirates', 'Afghanistan', 'Antigua and Barbuda', 'Anguilla', 'Albania', 'Armenia', 'Netherlands Antilles', 'Angola', 'Antarctica', 'Argentina', 'American Samoa', 'Austria', 'Australia', 'Aruba', 'Åland', 'Azerbaijan', 'Bosnia and Herzegovina', 'Barbados', 'Bangladesh', 'Belgium', 'Burkina Faso', 'Bulgaria', 'Bahrain', 'Burundi', 'Benin', 'Saint Barthélemy', 'Bermuda', 'Brunei', 'Bolivia', 'Bonaire', 'Brazil', 'Bahamas', 'Bhutan', 'Bouvet Island', 'Botswana', 'Belarus', 'Belize', 'Canada', 'Cocos [Keeling] Islands', 'Democratic Republic of the Congo', 'Central African Republic', 'Republic of the Congo', 'Switzerland', 'Ivory Coast', 'Cook Islands', 'Chile', 'Cameroon', 'China', 'Colombia', 'Costa Rica', 'Serbia and Montenegro', 'Cuba', 'Cape Verde', 'Curacao', 'Christmas Island', 'Cyprus', 'Czechia', 'Germany', 'Djibouti', 'Denmark', 'Dominica', 'Dominican Republic', 'Algeria', 'Ecuador', 'Estonia', 'Egypt', 'Western Sahara', 'Eritrea', 'Spain', 'Ethio

In [154]:
full_data[np.vectorize(lambda x: x in countries_with_nan_region)(full_data["FinalCountry"])]

,Key,DisplayName,Type,Region,Country,Url,WikiUrl,Location,ScrapedCountry,Method,FinalCountry
24950,aland university of applied sciences,Åland University of Applied Sciences,Education,NaN,Aland Islands,http://www.ha.ax/,http://en.wikipedia.org/wiki/Åland_University_...,Aland Islands,Aland Islands,Existing,Aland Islands


In [155]:
set(full_data["Region"])

{'AF', 'AS', 'EU', 'NA', 'OC', 'Other', 'SA', nan}

In [156]:
# Save data to use for mturk tasks
condition = np.vectorize(lambda x: x=="No data")
data["GoogleUrl"] = np.vectorize(lambda x: googlify_name(x, appendage="Headquarters"))(data["Key"])

mturk_data = data[condition(data["FinalCountry"])][["DisplayName", "Url","WikiUrl","GoogleUrl","FinalCountry"]]
mturk_data.to_csv("data_for_mturk.csv", index=False)
mturk_data

/Users/benjaminreadshaw/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,DisplayName,Url,WikiUrl,GoogleUrl,FinalCountry
346,Nomura Research Institute,http://www.nri.co.jp/english/,http://en.wikipedia.org/wiki/Nomura_Research_I...,https://www.google.com/search?q=nomura+researc...,No data
362,Rijkswaterstaat,http://www.rijkswaterstaat.nl/,http://en.wikipedia.org/wiki/Rijkswaterstaat,https://www.google.com/search?q=rijkswaterstaa...,No data
563,China Earthquake Networks Center,http://www.cenc.ac.cn/,http://en.wikipedia.org/wiki/China_Earthquake_...,https://www.google.com/search?q=china+earthqua...,No data
571,Telcordia Technologies,http://iconectiv.com/,http://en.wikipedia.org/wiki/Telcordia_Technol...,https://www.google.com/search?q=telcordia+tech...,No data
603,International Student Exchange Programs,http://www.isep.org/,http://th.wikipedia.org/wiki/index.html?curid=...,https://www.google.com/search?q=international+...,No data
784,Alcatel Vacuum Technology,https://www.pfeiffer-vacuum.com/,http://en.wikipedia.org/wiki/Adixen_Vacuum_Pro...,https://www.google.com/search?q=alcatel+vacuum...,No data
1138,Recife Center for Advanced Studies and Systems,http://www.cesar.org.br/,http://en.wikipedia.org/wiki/Recife_Center_for...,https://www.google.com/search?q=recife+center+...,No data
1142,Middle East Technical University Northern Cypr...,,,https://www.google.com/search?q=middle+east+te...,No data
1195,Paradigm,http://www.parabook.co.jp/,http://en.wikipedia.org/wiki/Paradigm_(publisher),https://www.google.com/search?q=paradigm+Headq...,No data
1269,SP Technical Research Institute of Sweden,http://www.ri.se/,http://en.wikipedia.org/wiki/SP_Technical_Rese...,https://www.google.com/search?q=sp+technical+r...,No data


In [157]:
## Combine results from Mturk with scraped results

mturk_results = dict()

with open("Mturk batch result - Batch_3804034_batch_results.csv", "r") as fh:
    reader = csv.reader(fh, delimiter=",")
    next(reader)
    for row in reader:
        institute = row[27]
        country = row[32]
        
        if institute not in mturk_results:
            mturk_results[institute] = {country: 1}
        
        else:
            if country in mturk_results[institute]:
                mturk_results[institute][country] += 1
            else:
                mturk_results[institute][country] = 1
                
print(len(mturk_results))

mturk_results = {inst: max(countries.items(), key=lambda x: x[1])[0] for inst, countries in mturk_results.items()
                    if len([k for k,v in countries.items() if v > sum(countries.values()) / 2]) }


print(len(mturk_results), mturk_results)

210
206 {'Indian Institutes of Science Education and Research': 'India', 'Zhejiang University City College': 'China', 'Rajamangala University of Technology': 'Thailand', 'Center for Complex Systems and Brain Sciences': 'United States', 'Kongsberg SIM': 'Norway', 'Indian Institutes of Information Technology': 'India', 'Fortune': 'United States', 'International Trademark Association': 'United States', "Los Angeles County Sheriff's Department": 'United States', 'Siemens IT Solutions and Services': 'Germany', 'Sesam': 'Norway', 'Catalan Institute of Nanotechnology': 'Spain', 'Los Angeles Biomedical Research Institute': 'United States', 'United States Atomic Energy Commission': 'United States', 'University of Science and Technology of Hanoi': 'Vietnam', 'Viktoria Institute': 'Sweden', 'Vaccine Research Center': 'United States', 'Okuma Corporation': 'Japan', 'Lockheed Martin Space Systems': 'United States', 'Fokus': 'Denmark', 'South African National Bioinformatics Institute': 'South Africa'

In [175]:
# assign mturk selected countries to FinalCountry
solved_with_mturk = np.vectorize(lambda inst: inst in mturk_results)
lookup_mturk_answer = np.vectorize(lambda inst: mturk_results[inst] if inst in mturk_results else "No data")
country_to_region["No data"] = "No data"
country_to_region["Aland Islands"] = "EU"
lookup_region = np.vectorize(lambda country: country_to_region[country])

# set final country for mturk results
full_data.loc[solved_with_mturk(full_data["DisplayName"]),"FinalCountry"] = lookup_mturk_answer(
    full_data[solved_with_mturk(full_data["DisplayName"])]["DisplayName"])

# set method used to fiund location as mturk
full_data.loc[solved_with_mturk(full_data["DisplayName"]),"Method"] = "Mturk"

full_data["FinalRegion"] = lookup_region(full_data["FinalCountry"])

In [177]:
full_data.to_csv("inst_full_clean.csv", 
                 columns=["Key","DisplayName","Type","FinalRegion","FinalCountry","Url"],
                 header=False,
                 index = False)

full_data.to_csv("inst_full_clean_with_scraping_info.csv", 
                 header=True,
                 index = False)

In [176]:
full_data[solved_with_mturk(full_data["DisplayName"])]

,Key,DisplayName,Type,Region,Country,Url,WikiUrl,Location,ScrapedCountry,Method,FinalCountry,FinalRegion
346,nomura research institute,Nomura Research Institute,Other,Other,,http://www.nri.co.jp/english/,http://en.wikipedia.org/wiki/Nomura_Research_I...,"Otemachi Financial City Grand Cube, 1-9-2 Otem...",ERROR_PROCESSING_LOCATION,Mturk,Japan,AS
362,rijkswaterstaat,Rijkswaterstaat,Other,Other,,http://www.rijkswaterstaat.nl/,http://en.wikipedia.org/wiki/Rijkswaterstaat,NO_LOCATION_FOUND,NO_COUNTRY_FOUND,Mturk,Netherlands,EU
563,china earthquake networks center,China Earthquake Networks Center,Other,Other,,http://www.cenc.ac.cn/,http://en.wikipedia.org/wiki/China_Earthquake_...,,ERROR_PROCESSING_LOCATION,Mturk,China,AS
571,telcordia technologies,Telcordia Technologies,Other,Other,,http://iconectiv.com/,http://en.wikipedia.org/wiki/Telcordia_Technol...,"Bridgewater, NJ U.S.A",ERROR_PROCESSING_LOCATION,Mturk,United States,NA
603,international student exchange programs,International Student Exchange Programs,Other,Other,,http://www.isep.org/,http://th.wikipedia.org/wiki/index.html?curid=...,NO_LOCATION_FOUND,NO_COUNTRY_FOUND,Mturk,United States,NA
784,alcatel vacuum technology,Alcatel Vacuum Technology,Other,Other,,https://www.pfeiffer-vacuum.com/,http://en.wikipedia.org/wiki/Adixen_Vacuum_Pro...,"Annecy, 98 avenue de Brogny, France",ERROR_PROCESSING_LOCATION,Mturk,France,EU
1138,recife center for advanced studies and systems,Recife Center for Advanced Studies and Systems,Other,Other,,http://www.cesar.org.br/,http://en.wikipedia.org/wiki/Recife_Center_for...,Recife Offices in Manaus Sorocaba Curitiba,ERROR_PROCESSING_LOCATION,Mturk,Brazil,SA
1142,middle east technical university northern cypr...,Middle East Technical University Northern Cypr...,Other,Other,,,,ODTÜ Kuzey Kampüsü Güzelyurt KKTC,ERROR_PROCESSING_LOCATION,Mturk,Turkey,AS
1195,paradigm,Paradigm,Other,Other,,http://www.parabook.co.jp/,http://en.wikipedia.org/wiki/Paradigm_(publisher),NO_LOCATION_FOUND,NO_COUNTRY_FOUND,Mturk,Japan,AS
1269,sp technical research institute of sweden,SP Technical Research Institute of Sweden,Other,Other,,http://www.ri.se/,http://en.wikipedia.org/wiki/SP_Technical_Rese...,NO_LOCATION_FOUND,NO_COUNTRY_FOUND,Mturk,Sweden,EU
